In [1]:
import json
import matplotlib
import numpy as np
import re
# from gensim.models import Word2Vec
# from gensim.models import KeyedVectors
# from gensim import corpora
# from collections import defaultdict

config = {
    # base info about dataset
    'train_num':44375,
    'test_num':21435,
    'test_num':21435,

    # paradim setting
    'sentence_len':8,
    'use_stopwords':True,

}
stoplist = set('a the this that these those of on in to for'.split())

def load_json_file():
# load all Q and A
    j_eval = open('./data/questions/val.json')
    j_train = open('./data/questions/train.json')
    j_test = open('./data/questions/test.json')

    Q_train = json.load(j_train)['questions']
    print("Q_train num {}".format(len(Q_train)))
    Q_eval = json.load(j_eval)['questions']
    print("Q_train num {}".format(len(Q_eval)))
    Q_test = json.load(j_test)['questions']
    print("Q_train num {}".format(len(Q_test)))

    j_eval = open('./data/annotations/val.json')
    j_train = open('./data/annotations/train.json')
    j_test = open('./data/annotations/test.json')

    # not checked ,uncertain about how to get the raw data
    A_train = json.load(j_train)['annotations']
    A_eval = json.load(j_eval)['annotations']
    A_test = json.load(j_test)['annotations']

    Q_set = Q_train+Q_eval+Q_test

    A_set = A_train+A_eval+A_test
    return Q_set,A_set

def get_vocab(Q_set,A_set):

    Vocab = []
    for i in range(len(Q_set)):
        Q = Q_set[i]['question']
        A = A_set[i]['multiple_choice_answer']
        Q = Q.lower()
        Q = re.sub(r"[^a-zA-Z]+", r"",Q)
        Q = Q.strip()
        A = A.lower()
        A = re.sub(r"[^a-zA-Z]+", r"",A)
        A = A.strip()

        Q_vocab = Q.split(' ')
        for word in Q_vocab:
            if(config['use_stopwords']):
                if word not in stoplist:
                    Vocab.append(word)
            else:
                Vocab.append(word)
        A_vocab = A.split(' ')
        for word in A_vocab:
            if(config['use_stopwords']):
                if word not in stoplist:
                    Vocab.append(word)
            else:
                Vocab.append(word)
    
    return Vocab

Q_train num 44375
Q_train num 21435
Q_train num 21435


In [96]:
Q_set,A_set = load_json_file()
Vocab = get_vocab(Q_set,A_set)

[{'image_id': 393223,
  'question': 'Is she brushing?',
  'question_id': 393223003},
 {'image_id': 393227,
  'question': 'What is this man riding on?',
  'question_id': 393227001},
 {'image_id': 131074,
  'question': 'Are the walls done in a summery color?',
  'question_id': 131074002},
 {'image_id': 393230,
  'question': 'What is the man doing?',
  'question_id': 393230003},
 {'image_id': 393230,
  'question': 'What is the person wearing?',
  'question_id': 393230005},
 {'image_id': 393230,
  'question': 'Did someone forget his luggage in the snow?',
  'question_id': 393230006},
 {'image_id': 131087,
  'question': 'What is in the water?',
  'question_id': 131087004},
 {'image_id': 131075,
  'question': 'What is on the display?',
  'question_id': 131075010},
 {'image_id': 524311,
  'question': 'What is the man playing?',
  'question_id': 524311000},
 {'image_id': 25, 'question': 'Where is the giraffe?', 'question_id': 25004},
 {'image_id': 30,
  'question': 'What is the vase sitting on

In [2]:
import os
def get_all_img_id():
    '''
    返回有重复的id和全部的id
    不要排序
    '''
    path_train = "C:\\Users\\lrn\\Desktop\\资料\\data\\data\\images\\train"
    path_eval = "C:\\Users\\lrn\\Desktop\\资料\\data\\data\\images\\val"
    path_test = "C:\\Users\\lrn\\Desktop\\资料\\data\\data\\images\\test"
    train_img = os.listdir(path_train)
    eval_img = os.listdir(path_eval)
    test_img = os.listdir(path_test)
    img_id = []
    for item in train_img+eval_img+test_img:
        unit = item.split('_')
        id = unit[2]
        id = id.split('.')[0]
        while id[0]=='0':
            id=id[1:]
        img_id.append(int(id))
    return img_id

def get_question_img_id(Q_set):
    Q_img_id = []
    for item in Q_set:
        id = item['image_id']
        # if id not in Q_img_id:
        Q_img_id.append(id)
    return Q_img_id

def get_answer_img_id(A_set):
    A_img_id = []
    for item in A_set:
        id = item['image_id']
        # if id not in Q_img_id:
        A_img_id.append(id)
    return A_img_id

def get_answer_Q_id(A_set):
    A_Q_id = []
    for item in A_set:
        id = item['question_id']
        A_Q_id.append(id)
    return A_Q_id

def get_Q_id(Q_set):
    Q_id = []
    for item in Q_set:
        id = item['question_id']
        # if id not in Q_img_id:
        Q_id.append(id)
    return Q_id


def check_QA_dataset_for_img(Q_img_id,img_id):
    '''
    返回没用的问题的索引（数组下角标）
    返回没用的图片id
    '''
    index = []
    img_nowhere = []
    # img_nouse = []
    for i,item in enumerate(Q_img_id):
        if item not in img_id:
            index.append(i)
            img_nowhere.append(item)

    print("{} questions/anwser have no corrsponding img .{}".format(len(img_nowhere),len(img_nowhere)/len(Q_img_id)))
    return index

def check_QA_corres(Q_id,A_Q_id):
    '''
    检查answer里面是不是回答了不存在的问题
    检查是不是有的Q没有A回答
    '''
    Q_index = []
    A_index = []
    for i,item in enumerate(Q_id):
        '''
        是不是有的问题没有人回答
        '''
        if item not in A_Q_id:
            Q_index.append(i)
    
    for i,item in enumerate(A_Q_id):
        if item not in Q_id:
            A_index.append(i)
    
    return Q_index,A_index



In [3]:
train_num = 44375
eval_num = 21435
test_num = 21435
img_id = get_all_img_id()
Q_img_id = get_question_img_id(Q_set)
A_img_id = get_answer_img_id(A_set)
A_Q_id = get_answer_Q_id(A_set)
Q_index = check_QA_dataset_for_img(Q_img_id,img_id)
A_index = check_QA_dataset_for_img(A_img_id,img_id)
Q_id = get_Q_id(Q_set)
# QA匹配，不用额外筛选
QA_Q_index,QA_A_index = check_QA_corres(Q_id,A_Q_id)




21304 questions/anwser have no corrsponding img .0.24418591323285002
21304 questions/anwser have no corrsponding img .0.24418591323285002


In [4]:
train_num = 44375
eval_num = 21435
test_num = 21435

# train set
train_Q_set = []
eval_Q_set = []
test_Q_set = []

for i,item in enumerate(Q_set):
    if i not in Q_index:
        if i < train_num:
            train_Q_set.append(item)
        elif i < train_num+eval_num:
            eval_Q_set.append(item)
        else:
            test_Q_set.append(item)



In [6]:
path = "./clean_data/questions/"
train_name = "train.json"
eval_name = "val.json"
test_name = "test.json"

file1 = open(path+train_name,mode='w')
file2 = open(path+eval_name,mode='w')
file3 = open(path+test_name,mode='w')

json.dump(train_Q_set,file1)
json.dump(eval_Q_set,file2)
json.dump(test_Q_set,file3)

file1.close()
file2.close()
file3.close()



In [7]:
test = open('./clean_data/questions/train.json')
test_read = json.load(test)
test.close()
test_read

[{'image_id': 131074,
  'question': 'Are the walls done in a summery color?',
  'question_id': 131074002},
 {'image_id': 393230,
  'question': 'What is the man doing?',
  'question_id': 393230003},
 {'image_id': 393230,
  'question': 'What is the person wearing?',
  'question_id': 393230005},
 {'image_id': 393230,
  'question': 'Did someone forget his luggage in the snow?',
  'question_id': 393230006},
 {'image_id': 25, 'question': 'Where is the giraffe?', 'question_id': 25004},
 {'image_id': 262187,
  'question': 'Is the plane taking off?',
  'question_id': 262187003},
 {'image_id': 49,
  'question': 'What color horse is closer to the camera?',
  'question_id': 49001},
 {'image_id': 49, 'question': 'Are they playing polo?', 'question_id': 49002},
 {'image_id': 262204,
  'question': 'What color is the animal?',
  'question_id': 262204002},
 {'image_id': 458763,
  'question': 'What are the vases sitting on?',
  'question_id': 458763000},
 {'image_id': 524377,
  'question': 'What kind of

In [8]:
train_num = 44375
eval_num = 21435
test_num = 21435

# train set
train_A_set = []
eval_A_set = []
test_A_set = []

for i,item in enumerate(A_set):
    if i not in A_index:
        if i < train_num:
            train_A_set.append(item)
        elif i < train_num+eval_num:
            eval_A_set.append(item)
        else:
            test_A_set.append(item)

path = "./clean_data/annotations/"
train_name = "train.json"
eval_name = "val.json"
test_name = "test.json"

file11 = open(path+train_name,mode='w')
file12 = open(path+eval_name,mode='w')
file13 = open(path+test_name,mode='w')

json.dump(train_A_set,file11)
json.dump(eval_A_set,file12)
json.dump(test_A_set,file13)

file11.close()
file12.close()
file13.close()


In [9]:
test = open('./clean_data/annotations/train.json')
test_read = json.load(test)
test.close()
test_read

[{'question_type': 'what is the',
  'multiple_choice_answer': 'brick',
  'answers': [{'answer': 'brick', 'answer_confidence': 'yes', 'answer_id': 1},
   {'answer': 'brick', 'answer_confidence': 'yes', 'answer_id': 2},
   {'answer': 'brick', 'answer_confidence': 'yes', 'answer_id': 3},
   {'answer': 'bricks', 'answer_confidence': 'yes', 'answer_id': 4},
   {'answer': 'brick', 'answer_confidence': 'yes', 'answer_id': 5},
   {'answer': 'brick', 'answer_confidence': 'yes', 'answer_id': 6},
   {'answer': 'brick', 'answer_confidence': 'yes', 'answer_id': 7},
   {'answer': 'brick', 'answer_confidence': 'yes', 'answer_id': 8},
   {'answer': 'brick', 'answer_confidence': 'yes', 'answer_id': 9},
   {'answer': 'brick', 'answer_confidence': 'yes', 'answer_id': 10}],
  'image_id': 118989,
  'answer_type': 'other',
  'question_id': 118989008},
 {'answer_type': 'other',
  'multiple_choice_answer': 'concrete',
  'answers': [{'answer': 'cement', 'answer_confidence': 'yes', 'answer_id': 1},
   {'answer'